In [2]:
from manim import *

class Eigenvector3DScene(ThreeDScene):
    """
    A deeper animation explaining eigenvectors in 3D.
    
    This scene:
    1. Includes 3D grid planes and a unit cube.
    2. Labels vectors with 2D text that follows the 3D arrow tips.
    3. Animates the coordinate labels changing during the transform.
    4. Isolates the eigenvector to show the A*v = lambda*v math.
    5. Isolates the non-eigenvector to show why it fails the test.
    """
    def construct(self):
        # --- 1. Setup Scene ---
        
        # Define ranges for axes and planes
        x_range = [-5, 5, 1]
        y_range = [-5, 5, 1]
        z_range = [-5, 5, 1]
        x_length = 10
        y_length = 10
        z_length = 10

        axes = ThreeDAxes(
            x_range=x_range,
            y_range=y_range,
            z_range=z_range,
            x_length=x_length,
            y_length=y_length,
            z_length=z_length,
        )
        
        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES, zoom=0.75)
        
        title = Text("What is an Eigenvector in 3D?").to_corner(UL)
        self.add_fixed_in_frame_mobjects(title)

        matrix = [
            [2, 0, 0],
            [0, 0.5, 0],
            [0, 0, 1]
        ]
        
        matrix_tex = MathTex(
            r"A = \begin{pmatrix} 2 & 0 & 0 \\ 0 & 0.5 & 0 \\ 0 & 0 & 1 \end{pmatrix}",
            font_size=36
        ).to_corner(UR)
        self.add_fixed_in_frame_mobjects(matrix_tex)
        
        # --- Add Grid Planes ---
        grid_plane_config = {
            "stroke_width": 1,
            "stroke_opacity": 0.2,
            "faded_line_ratio": 2, # Makes major lines stand out
        }

        xy_plane = NumberPlane(
            x_range=x_range, y_range=y_range,
            x_length=x_length, y_length=y_length,
            **grid_plane_config
        )
        xz_plane = NumberPlane(
            x_range=x_range, y_range=z_range,
            x_length=x_length, y_length=z_length,
            **grid_plane_config
        ).rotate(PI/2, axis=RIGHT) # Rotate around X-axis

        yz_plane = NumberPlane(
            x_range=y_range, y_range=z_range,
            x_length=y_length, y_length=z_length,
            **grid_plane_config
        ).rotate(PI/2, axis=UP) # Rotate around Y-axis

        self.add(axes, xy_plane, xz_plane, yz_plane)
        self.wait(1)

        # --- 2. Create Vectors and Coordinate Labels ---

        v_eigen = Arrow(ORIGIN, [2, 0, 0], buff=0, color=GREEN)
        eigen_span = DashedLine(axes.c2p(-5, 0, 0), axes.c2p(5, 0, 0), color=GREEN, stroke_opacity=0.5)
        
        v_other = Arrow(ORIGIN, [1, 2, 1], buff=0, color=RED)
        other_span = DashedLine(axes.c2p(-2, -4, -2), axes.c2p(2, 4, 2), color=RED, stroke_opacity=0.5)

        unit_cube = Cube(
            side_length=1,
            fill_opacity=0.1,
            stroke_width=2,
            stroke_color=WHITE,
            fill_color=BLUE
        ).move_to(axes.c2p(1.5, 1.5, 0.5))

        # --- THIS IS THE NEW LOGIC ---
        # Create "Before" and "After" labels
        v_eigen_label_pre = MathTex(r"\vec{v}_{\lambda} = \begin{bmatrix} 2 \\ 0 \\ 0 \end{bmatrix}", color=GREEN, font_size=36)
        v_other_label_pre = MathTex(r"\vec{w} = \begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix}", color=RED, font_size=36)
        
        v_eigen_label_post = MathTex(r"A\vec{v}_{\lambda} = \begin{bmatrix} 4 \\ 0 \\ 0 \end{bmatrix}", color=GREEN, font_size=36)
        v_other_label_post = MathTex(r"A\vec{w} = \begin{bmatrix} 2 \\ 1 \\ 1 \end{bmatrix}", color=RED, font_size=36)

        # Define Updaters that tie 2D labels to 3D arrow tips
        def eigen_label_updater(m):
            # 1. Get 3D tip position
            tip_point_3d = v_eigen.get_end()
            # 2. Project to 2D screen coordinates
            # --- THIS IS THE FIX ---
            screen_coords_2d = self.camera.project_point(tip_point_3d)
            # 3. Move the label (m) to that 2D point, then shift
            m.move_to(screen_coords_2d).shift(RIGHT * 1.5) # Shift right in screen space

        def other_label_updater(m):
            tip_point_3d = v_other.get_end()
            # 2. Project to 2D screen coordinates
            # --- THIS IS THE FIX ---
            screen_coords_2d = self.camera.project_point(tip_point_3d)
            # 3. Move the label (m) to that 2D point, then shift
            m.move_to(screen_coords_2d).shift(UP * 1.0) # Shift up in screen space

        # Add updaters to all four labels
        v_eigen_label_pre.add_updater(eigen_label_updater)
        v_eigen_label_post.add_updater(eigen_label_updater)
        v_other_label_pre.add_updater(other_label_updater)
        v_other_label_post.add_updater(other_label_updater)

        # Add 3D objects to scene
        self.add(v_eigen, eigen_span, v_other, other_span, unit_cube)
        # Add "Before" labels to the fixed frame
        self.add_fixed_in_frame_mobjects(v_eigen_label_pre, v_other_label_pre)
        self.wait(2)

        # --- 3. Apply Transformation and Animate Labels ---
        
        explanation = Text(
            "Applying transformation A to the whole space...",
            font_size=28
        ).to_corner(DL)
        self.add_fixed_in_frame_mobjects(explanation)
        self.wait(2)
        self.remove_fixed_in_frame_mobjects(explanation)

        transform_group = VGroup(
            v_eigen, v_other, axes, eigen_span, other_span,
            xy_plane, xz_plane, yz_plane, unit_cube
        )

        self.play(
            ApplyMatrix(matrix, transform_group),
            ReplacementTransform(v_eigen_label_pre, v_eigen_label_post),
            ReplacementTransform(v_other_label_pre, v_other_label_post),
            run_time=4
        )
        self.wait(1)

        # --- 4. Deeper Analysis: Eigenvector (Green) ---
        
        analysis_group_red_and_cube = VGroup(v_other, other_span, unit_cube)
        # Fade out 3D red objects
        self.play(FadeOut(analysis_group_red_and_cube))
        # Fade out 2D red label (which is now v_other_label_post)
        self.play(FadeOut(v_other_label_post))
        self.remove_fixed_in_frame_mobjects(v_other_label_post)
        
        # ... math text logic remains the same ...
        math_step1 = MathTex(
            r"A\vec{v}_{\lambda} = \begin{pmatrix} 2 & 0 & 0 \\ 0 & 0.5 & 0 \\ 0 & 0 & 1 \end{pmatrix} \begin{bmatrix} 2 \\ 0 \\ 0 \end{bmatrix} = \begin{bmatrix} 4 \\ 0 \\ 0 \end{bmatrix}",
            font_size=36, color=GREEN
        ).to_corner(DL)
        self.add_fixed_in_frame_mobjects(math_step1)
        self.play(Write(math_step1))
        self.wait(2)
        
        math_step2 = MathTex(
            r"A\vec{v}_{\lambda} = \begin{bmatrix} 4 \\ 0 \\ 0 \end{bmatrix} = 2 \times \begin{bmatrix} 2 \\ 0 \\ 0 \end{bmatrix}",
            font_size=36, color=GREEN
        ).to_corner(DL)
        
        self.play(FadeOut(math_step1))
        self.remove_fixed_in_frame_mobjects(math_step1)
        self.add_fixed_in_frame_mobjects(math_step2)
        self.play(FadeIn(math_step2))
        self.wait(2)

        math_step3 = MathTex(
            r"A\vec{v}_{\lambda} = 2 \cdot \vec{v}_{\lambda}",
            r" \text{ (This is an Eigenvector!) }",
            font_size=36
        )
        math_step3[0].set_color(GREEN)
        math_step3.to_corner(DL)

        self.play(FadeOut(math_step2))
        self.remove_fixed_in_frame_mobjects(math_step2)
        self.add_fixed_in_frame_mobjects(math_step3)
        self.play(FadeIn(math_step3))
        self.wait(3)
        self.remove_fixed_in_frame_mobjects(math_step3)

        # --- 5. Deeper Analysis: Non-Eigenvector (Red) ---
        
        analysis_group_green = VGroup(v_eigen, eigen_span)
        # Fade out 3D green objects
        self.play(FadeOut(analysis_group_green))
        # Fade out 2D green label
        self.play(FadeOut(v_eigen_label_post))
        self.remove_fixed_in_frame_mobjects(v_eigen_label_post) 

        # Fade in 3D red objects
        self.play(FadeIn(analysis_group_red_and_cube))
        # Fade in 2D red label
        self.add_fixed_in_frame_mobjects(v_other_label_post) 
        self.play(FadeIn(v_other_label_post))

        # ... math text logic remains the same ...
        w_math_step1 = MathTex(
            r"A\vec{w} = \begin{pmatrix} 2 & 0 & 0 \\ 0 & 0.5 & 0 \\ 0 & 0 & 1 \end{pmatrix} \begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix} = \begin{bmatrix} 2 \\ 1 \\ 1 \end{bmatrix}",
            font_size=36, color=RED
        ).to_corner(DL)
        self.add_fixed_in_frame_mobjects(w_math_step1)
        self.play(Write(w_math_step1))
        self.wait(2)

        w_math_line1 = MathTex(
            r"A\vec{w} = \begin{pmatrix} 2 & 0 & 0 \\ 0 & 0.5 & 0 \\ 0 & 0 & 1 \end{pmatrix} \begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix} = \begin{bmatrix} 2 \\ 1 \\ 1 \end{bmatrix}",
            font_size=36, color=RED
        )
        w_math_line2 = MathTex(r"\text{Is } \begin{bmatrix} 2 \\ 1 \\ 1 \end{bmatrix} = c \begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix} \text{ for any scalar } c \text{?}", font_size=32)
        w_math_line3 = Text("No. The direction changed.", color=RED, font_size=28)
        
        w_math_group = VGroup(w_math_line1, w_math_line2, w_math_line3).arrange(DOWN, aligned_edge=LEFT).to_corner(DL)
        
        self.play(Transform(w_math_step1, w_math_line1))
        
        self.add_fixed_in_frame_mobjects(w_math_line2) 
        self.play(Write(w_math_line2))
        self.wait(2)
        
        self.add_fixed_in_frame_mobjects(w_math_line3)
        self.play(Write(w_math_line3))
        self.wait(3)

        # --- 6. Final Conclusion ---
        
        # Bring back green 3D objects
        self.play(FadeIn(analysis_group_green))
        # Bring back green 2D label
        self.add_fixed_in_frame_mobjects(v_eigen_label_post) 
        self.play(FadeIn(v_eigen_label_post))
        
        final_def = Text(
            "The GREEN vector is an Eigenvector:\n"
            "It stayed on its span and only scaled.\n\n"
            "The RED vector is NOT:\n"
            "It was knocked off its span; its direction changed.",
            font_size=28,
            line_spacing=1.2
        ).to_corner(DL)
        
        # Fade out old math text
        self.play(
            FadeOut(w_math_step1), # This is the transformed w_math_line1
            FadeOut(w_math_line2),
            FadeOut(w_math_line3)
        )
        self.remove_fixed_in_frame_mobjects(w_math_step1, w_math_line2, w_math_line3)
        
        # Write final conclusion
        self.add_fixed_in_frame_mobjects(final_def)
        self.play(Write(final_def))
        
        # A final camera move to appreciate the 3D
        # The labels will now move with the camera!
        self.move_camera(phi=60 * DEGREES, theta=120 * DEGREES, run_time=3)
        self.wait(5)

%manim -qk -v warning Eigenvector3DScene

Manim Community v0.19.0